In [2]:
import os
import requests

# Specify the folder containing the images
folder_path = 'images'

# List all files in the folder and filter for image files (if needed)
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

# Prepare the files dictionary for the POST request
files = [('images', (os.path.basename(image_path), open(image_path, 'rb'))) for image_path in image_files]

# Make the POST request
upload_response = requests.post('http://localhost:8000/upload-images', files=files)

# Print the response
print(upload_response.status_code)
print(upload_response.json())


200
{'message': 'Upload successful, training started'}


In [15]:
import requests

# Make the GET request to the endpoint
progress_response = requests.get('http://localhost:8000/get-training-progress')

# Print the HTTP status code
print(f"Status Code: {progress_response.status_code}")

# Print the JSON response if the status code is 200 (OK)
if progress_response.status_code == 200:
    print("Training Progress:")
    print(progress_response.json())
else:
    print(f"Error: {progress_response.text}")


Status Code: 200
Training Progress:
{'progress': 0, 'status': 'training', 'time_metrics': {'elapsed_time': '02:10:56', 'estimated_time_remaining': None, 'training_duration': None}, 'training_metrics': {'current_epoch': 0, 'total_epochs': 0, 'current_step': 0, 'total_steps': 0, 'epoch_progress': 0.0, 'overall_progress': 0.0, 'current_loss': None}, 'resource_metrics': {'gpu_memory': {'timestamp': 1738151334.275956, 'used_memory': 23561.0, 'total_memory': 46068.0, 'utilization': 51.14396110098116}, 'cpu_utilization': {'timestamp': 1738151334.2761085, 'utilization': 26.3}}, 'validation_metrics': {'validation_loss': None, 'accuracy': None, 'f1_score': None, 'precision': None, 'recall': None}, 'error': None}


In [11]:
import requests

# Request the LoRa file status or URL
lora_response = requests.get('http://localhost:8000/get-lora-file')

# Print the HTTP status code
print(f"Status Code: {lora_response.status_code}")

# Check if the response is successful
if lora_response.status_code == 200:
    print("LoRa File Details:")
    print(lora_response.json())
else:
    print(f"Error: {lora_response.text}")


Status Code: 200
LoRa File Details:
{'status': 'in_progress', 'progress': 0, 'details': {'start_time': 1738148709.5298123, 'end_time': None, 'total_epochs': 0, 'current_epoch': 0, 'total_steps': 0, 'current_step': 0, 'current_loss': None, 'loss_history': [], 'epoch_progress': 0.0, 'overall_progress': 0.0, 'estimated_time_remaining': None, 'formatted_time_remaining': None, 'elapsed_time': 4501.570471048355, 'formatted_elapsed_time': '01:15:01', 'validation_metrics': {'validation_loss': None, 'accuracy': None, 'f1_score': None, 'precision': None, 'recall': None}, 'gpu_memory_usage': [{'timestamp': 1738148712.7445588, 'used_memory': 2121.0, 'total_memory': 46068.0, 'utilization': 4.604063558218286}, {'timestamp': 1738149762.0388422, 'used_memory': 14857.0, 'total_memory': 46068.0, 'utilization': 32.25015194929235}, {'timestamp': 1738150713.407215, 'used_memory': 19897.0, 'total_memory': 46068.0, 'utilization': 43.19050099852392}, {'timestamp': 1738150743.8338726, 'used_memory': 20157.0, '

In [13]:
import boto3
import sagemaker
from datetime import datetime

def check_sagemaker_default_bucket():
    """
    Check SageMaker default bucket details.
    
    Returns:
    dict: Comprehensive bucket information
    """
    try:
        # Get the SageMaker session and default bucket
        sagemaker_session = sagemaker.Session()
        default_bucket = sagemaker_session.default_bucket()
        
        # Create S3 clients
        s3_client = boto3.client('s3')
        s3_resource = boto3.resource('s3')
        
        # Get bucket metadata
        bucket = s3_resource.Bucket(default_bucket)
        
        # List objects in the bucket
        try:
            response = s3_client.list_objects_v2(Bucket=default_bucket)
            
            # Get bucket creation time
            bucket_info = s3_client.head_bucket(Bucket=default_bucket)
            creation_date = bucket_info.get('ResponseMetadata', {}).get('HTTPHeaders', {}).get('date')
            
            # Check if bucket is empty
            if response['KeyCount'] == 0:
                return {
                    'bucket_name': default_bucket,
                    'exists': True,
                    'is_empty': True,
                    'creation_date': creation_date,
                    'total_objects': 0,
                    'total_size_bytes': 0
                }
            
            # Calculate total size
            total_size = sum(obj.size for obj in bucket.objects.all())
            
            return {
                'bucket_name': default_bucket,
                'exists': True,
                'is_empty': False,
                'creation_date': creation_date,
                'total_objects': response['KeyCount'],
                'total_size_bytes': total_size,
                'objects': [
                    {
                        'key': obj['Key'], 
                        'size': s3_client.head_object(Bucket=default_bucket, Key=obj['Key'])['ContentLength']
                    } 
                    for obj in response.get('Contents', [])
                ]
            }
        
        except s3_client.exceptions.NoSuchBucket:
            return {
                'exists': False,
                'error': 'Bucket does not exist'
            }
    
    except Exception as e:
        return {
            'exists': False,
            'error': str(e)
        }

# Example usage
bucket_details = check_sagemaker_default_bucket()
print("Bucket Details:")
for key, value in bucket_details.items():
    print(f"{key}: {value}")

Bucket Details:
bucket_name: sagemaker-us-east-1-274412008471
exists: True
is_empty: False
creation_date: Wed, 29 Jan 2025 08:25:40 GMT
total_objects: 1
total_size_bytes: 343805440
objects: [{'key': 'trained_models/vijay_field/vijay_field_000001800.safetensors', 'size': 343805440}]


In [3]:
pip install sagemaker


Note: you may need to restart the kernel to use updated packages.


In [4]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket() 

NameError: name 'sagemaker' is not defined